In [1]:
from ultralytics import YOLO
import os
from collections import defaultdict

In [2]:

# Function to convert a single annotation to YOLO format
def convert_to_yolo(annotation, image_width, image_height):
    # Adjust class_id to start from 0 (assuming category_id starts from 1)
    class_id = annotation['category_id']
    x = annotation['0']  # Top-left x coordinate
    y = annotation['1']  # Top-left y coordinate
    width = annotation['2']  # Width of bounding box
    height = annotation['3']  # Height of bounding box
    
    # Calculate normalized center coordinates and dimensions
    center_x = (x + width / 2) / image_width
    center_y = (y + height / 2) / image_height
    width_norm = width / image_width
    height_norm = height / image_height
    
    # Return YOLO format string
    return f"{class_id} {center_x} {center_y} {width_norm} {height_norm}"


In [3]:

# Function to create YOLO annotation files
def create_yolo_annotations(annotations, image_info_list, output_dir):
    # Create a dictionary from image_id to image info
    image_dict = {img['id']: img for img in image_info_list}
    
    # Group annotations by image_id
    annotations_by_image = defaultdict(list)
    for ann in annotations:
        annotations_by_image[ann['image_id']].append(ann)
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    for image_id, anns in annotations_by_image.items():
        image_info = image_dict[image_id]
        image_width = image_info['width']
        image_height = image_info['height']
        image_file_name = image_info['file_name']
        
        # Define annotation file path
        ann_file_name = os.path.splitext(image_file_name)[0] + '.txt'
        ann_file_path = os.path.join(output_dir, ann_file_name)
        
        # Write annotations to file
        with open(ann_file_path, 'w') as f:
            for ann in anns:
                yolo_line = convert_to_yolo(ann, image_width, image_height)
                f.write(yolo_line + '\n')


In [4]:

# Example data (replace with your actual data)
annotations = [
    {
        'id': 0,
        'image_id': 9146,
        'category_id': 1,
        'area': 3861.0041370397084,
        '0': 2.9999542236328125,
        '1': 315.9999918937683,
        '2': 99.00009155273438,
        '3': 39.0000057220459,
        'iscrowd': 0
    }
    # Add more annotations as needed
]

images = [
    {'id': 9146, 'file_name': 'image9146.jpg', 'width': 640, 'height': 480}
    # Add more image info as needed
]


In [ ]:

# Convert annotations to YOLO format
create_yolo_annotations(annotations, images, 'data/train/labels')

# Load a pre-trained YOLOv11 model
model = YOLO('yolo11n.pt')  # Use 'yolo11s.pt', 'yolo11m.pt', etc., for other variants

# Define the path to your dataset YAML file
dataset_yaml = 'data/dataset.yaml'


In [ ]:

# Train the model
model.train(
    data=dataset_yaml,
    epochs=100,  # Adjust based on your needs
    imgsz=640,   # Image size (adjust if necessary)
    batch=16,    # Batch size (adjust based on GPU memory)
    name='yolo11_custom'  # Name for this training run
)



In [ ]:
# Evaluate the model on the validation set
model.val()


In [ ]:

# Save the trained model
model.save('yolo11_custom.pt')


In [ ]:

# Example inference
results = model('data/train/images/image9146.jpg')
results.show()